In [4]:
import os
import tqdm as notebook_tqdm
from dotenv import load_dotenv

import dsp
import dspy
from dspy.datasets.gsm8k import GSM8K, gsm8k_metric
from dspy.evaluate import Evaluate
from dspy.teleprompt import BootstrapFewShot, BootstrapFinetune

load_dotenv()
unify_api_key = os.getenv("UNIFY_KEY")



In [5]:
from dspy.datasets import HotPotQA

# Load the dataset.
dataset = HotPotQA(train_seed=1, train_size=20, eval_seed=2023, dev_size=50, test_size=0)

# Tell DSPy that the 'question' field is the input. Any other fields are labels and/or metadata.
trainset = [x.with_inputs('question') for x in dataset.train]
devset = [x.with_inputs('question') for x in dataset.dev]

len(trainset), len(devset)

Generating test split: 100%|██████████| 7405/7405 [00:02<00:00, 3202.32 examples/s]


(20, 50)

In [ ]:
class GenerateAnswer(dspy.Signature):
    """Answer questions with short factoid answers."""

    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

class RAG(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()

        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)

    def forward(self, question):
        context = self.retrieve(question).passages
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer)

In [ ]:
gemma = dsp.Unify(
    endpoint="gemma-7b-it@anyscale",
    max_tokens=150,
    api_key=unify_api_key,
)

claude = dsp.Unify(
    endpoint="claude-3-haiku@anthropic",
    max_tokens=150,
    api_key=unify_api_key,
)

unify_router = dsp.Unify(
    max_tokens=150,
    api_key=unify_api_key,
)

In [ ]:
def validate_context_and_answer(example, pred, trace=None):
    answer_EM = dspy.evaluate.answer_exact_match(example, pred)
    return answer_EM

num_threads = 24

teleprompter = BootstrapFewShot(metric=validate_context_and_answer)

compiled_rag = teleprompter.compile(RAG(), trainset=trainset)

In [ ]:
evaluate_hotpot = Evaluate(devset=devset[:1000], metric=validate_context_and_answer, num_threads=num_threads, display_progress=True, display_table=0)
evaluate_hotpot(gemma)